# Performance Test

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mlearn.functional as F
import mlearn as mlearn
import time
import torch
from time import time
import sys
from ctypes import POINTER, c_double

In [2]:
EPOCHS = 500
_inputs = np.random.randn(EPOCHS,32,3,32,32).astype(np.float32)
_weights = np.random.randn(8,3,5,5).astype(np.float32)
_bias = np.ones((8))

In [3]:
# 3.15
# 4.29
inputs = mlearn.tensor(_inputs)
weights = mlearn.tensor(_weights)
bias = mlearn.tensor(_bias)

start = time()
for e,batch in enumerate(inputs):
    c_out = F.conv_2d(batch,weights,bias,stride=1)
#     print(f"\r{e+1}/{EPOCHS}",end="")
#     sys.stdout.flush()
# print()
mlearn_conv = (time() - start)
# ptr = (c_double*24*24*8).from_address(int(c_out[0].data))
# c_out = np.ctypeslib.as_array(ptr)
print(c_out[0,0,0])

Tensor([  2.6658,  15.3152,   6.2044,  -4.0168, -12.0204, -13.1985,
          8.8064,   6.5382,  -0.1669, -15.4537,   9.3900, -17.1896,
          1.7938,   2.8899,  -3.3070,   7.1419,  12.8133,   1.9819,
         -0.7478,  -6.2937,  -9.4049,  -2.5398,  -4.9162, -20.8104,
         -6.0235,   2.9436,  13.9301,   0.3247])


In [4]:
inputs = torch.tensor(_inputs)
w = torch.tensor(_weights)
b = torch.tensor(_bias)
start = time()
# for e,batch in enumerate(inputs):
for e,batch in enumerate(inputs):
    out = torch.nn.functional.conv2d(batch.double(),w.double(),b.double())
#     print(f"\r{e+1}/{EPOCHS}",end="")
#     sys.stdout.flush()
# print()
torch_conv = (time() - start)
print(out[0,0,0])

tensor([  2.6658,  15.3152,   6.2044,  -4.0168, -12.0204, -13.1985,   8.8064,
          6.5382,  -0.1669, -15.4537,   9.3900, -17.1896,   1.7938,   2.8899,
         -3.3070,   7.1419,  12.8133,   1.9819,  -0.7478,  -6.2937,  -9.4049,
         -2.5398,  -4.9162, -20.8104,  -6.0235,   2.9436,  13.9301,   0.3247],
       dtype=torch.float64)


In [5]:
print("mlearn -> %.5f"%mlearn_conv)
print("torch  -> %.5f"%torch_conv)
print("比Pytorch慢 %.5f倍"%(mlearn_conv/torch_conv))

mlearn -> 1.57137
torch  -> 0.73913
比Pytorch慢 2.12597倍


In [6]:
# 无OpenMP
# mlearn -> 8.89202
# torch  -> 4.54154
# 比Pytorch慢 1.95793倍

In [7]:
# 有OpenMP
# mlearn -> 8.57620
# torch  -> 5.01696
# 比Pytorch慢 1.70944倍

In [8]:
# mlearn -> 5.21169
# torch  -> 2.23837
# 比Pytorch慢 2.32834倍